# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 1784, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 1784 (delta 98), reused 61 (delta 29), pack-reused 1618 (from 1)
Receiving objects: 100% (1784/1784), 69.71 MiB | 23.71 MiB/s, done.
Resolving deltas: 100% (1043/1043), done.


In [3]:
!pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 34.3 MB/s eta 0:00:00


In [4]:
!cd RECsys_Challenge2024 && python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
M

In [5]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil

np.random.seed(42)

## Import the repository

In [6]:
from github import Github, Auth

# Authenticate using a personal access token
auth_token = Auth.Token(token)
github_client = Github(auth=auth_token)

# Define the repository name you want to find
target_repo_name = 'RECsys_Challenge2024'
repo = None

# Search for the repository in the user's repositories
try:
    for repository in github_client.get_user().get_repos():
        if repository.name == target_repo_name:
            repo = repository
            print(f"Repository '{target_repo_name}' found.")
            break
    if repo is None:
        print(f"Repository '{target_repo_name}' not found.")
except Exception as e:
    print("An error occurred while accessing the repositories:", e)

Repository 'RECsys_Challenge2024' found.


In [7]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    """
    Uploads a file from Kaggle to GitHub, updating it if it already exists in the repository,
    or creating it if it does not.

    Parameters:
    - filepath_kaggle: Path to the file in the Kaggle environment.
    - filepath_github: Target path in the GitHub repository where the file should be uploaded.
    - commit_message: Message for the commit on GitHub.
    """
    try:
        
        # Check if the file already exists in the GitHub repository
        contents = repo.get_contents(filepath_github)
        
        # If it exists, update the file
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(
                contents.path, commit_message, file.read(), contents.sha
            )
        print(f"File '{filepath_github}' updated successfully.")
    
    except Exception as e:
        
        # If the file does not exist, create it
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(
                filepath_github, commit_message, file.read()
            )
        print(f"File '{filepath_github}' created successfully.")

In [39]:
config = {
    'model': 'RP3beta',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_RP3betaRecommender.db',
    'copy_prev_best_params': False,
    'save_github': True,
    'fixed' : True
}

Import the database where previous tuning trials have been saved.

In [40]:
try:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/' \
        f'{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [11]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [12]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [13]:
cd /kaggle/working/RECsys_Challenge2024/

/kaggle/working/RECsys_Challenge2024


In [14]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [44]:
import optuna
import pandas as pd
# from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

def objective_function_RP3betaRecommender(optuna_trial):
    
    recommender_instance = RP3betaRecommender(URM_train)
    
    

    full_hyperp = {

# {'topK': 14, 'alpha': 1.3115516327088887, 'normalize_similarity': True, 'implicit': False}
        # {"alpha": 0.369533359594755, "normalize_similarity": true, "implicit": true}
                   # "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 3, log=False),
                   "beta": optuna_trial.suggest_float("beta", 0, 3, log=False),
    
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),
                   "topK": 10  # Fixed value, not tuned
    #                 "alpha": 0.369533359594755,
    #                 "normalize_similarity": True,
    #                 "implicit": True
    }

    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [45]:
if config['tune_parameters']:
    
    study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    study.optimize(objective_function_RP3betaRecommender, n_trials=30)

[I 2024-11-15 22:37:19,750] A new study created in RDB with name: hyperparameters_tuning_RP3beta


RP3betaRecommender: Similarity column 38121 (100.0%), 2146.37 column/sec. Elapsed time 17.76 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 32.31 sec. Users per second: 1102


[I 2024-11-15 22:38:11,393] Trial 0 finished with value: 0.01367078486589691 and parameters: {'alpha': 1.7575884479692978, 'beta': 0.978229592098218, 'normalize_similarity': False, 'implicit': True}. Best is trial 0 with value: 0.01367078486589691.


RP3betaRecommender: Similarity column 38121 (100.0%), 2130.27 column/sec. Elapsed time 17.89 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 32.05 sec. Users per second: 1111


[I 2024-11-15 22:39:02,916] Trial 1 finished with value: 0.011259298278472543 and parameters: {'alpha': 2.066275500083099, 'beta': 0.8856867039648911, 'normalize_similarity': False, 'implicit': False}. Best is trial 0 with value: 0.01367078486589691.


...

[I 2024-11-15 23:01:13,400] Trial 27 finished with value: 0.053731276909319704 and parameters: {'alpha': 0.5641307833488952, 'beta': 0.28146845879960086, 'normalize_similarity': True, 'implicit': True}. Best is trial 27 with value: 0.053731276909319704.


RP3betaRecommender: Similarity column 38121 (100.0%), 2163.79 column/sec. Elapsed time 17.62 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 33.41 sec. Users per second: 1066


[I 2024-11-15 23:02:06,099] Trial 28 finished with value: 0.047324899609137515 and parameters: {'alpha': 0.5266720702014889, 'beta': 0.6504907829362314, 'normalize_similarity': True, 'implicit': True}. Best is trial 27 with value: 0.053731276909319704.


RP3betaRecommender: Similarity column 38121 (100.0%), 2143.06 column/sec. Elapsed time 17.79 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 32.80 sec. Users per second: 1085


[I 2024-11-15 23:02:58,191] Trial 29 finished with value: 0.028965315264092448 and parameters: {'alpha': 0.2527401984085997, 'beta': 1.1042048190913074, 'normalize_similarity': False, 'implicit': True}. Best is trial 27 with value: 0.053731276909319704.


In [47]:
recommender_instance = RP3betaRecommender(URM_train + URM_validation)
recommender_instance.fit(**study.best_trial.params)
# {"alpha": 0.369533359594755, "normalize_similarity": true, "implicit": true}
# recommender_instance.fit(topK=10, alpha=0.369533359594755, normalize_similarity=True, implicit=True)


RP3betaRecommender: Similarity column 38121 (100.0%), 1396.82 column/sec. Elapsed time 27.29 sec


# Testing

Create the recommendations for the submission. 

In [48]:
def create_submission(data_target_users_test, recommender_instance, cutoff=10, output_file=f"/kaggle/working/submission_{config['model']}.csv"):
    
    target_result = []

    for target in data_target_users_test["user_id"]:
        target_result.append(recommender_instance.recommend(target, cutoff=cutoff, remove_seen_flag=True))

    user_ids = data_target_users_test["user_id"]
    formatted_data = {
        "user_id": user_ids,
        "item_list": [" ".join(map(str, items)) for items in target_result]
    }

    submission_df = pd.DataFrame(formatted_data)
    submission_df.to_csv(output_file, index=False, header=["user_id", "item_list"])

    print(f"Submission file saved as {output_file}")

In [49]:
data_target_users_test = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv")

create_submission(data_target_users_test, recommender_instance)

# upload_file(
#             f'/kaggle/working/submission_{config["model"]}_fixed_topk.csv', 
#             f'/kaggle/working/Recsys_Challenge_2023/TrainedModels/{config["model"]}Recommender/'\
#                 f'submission_{config["model"]}.csv', 
#             f'New {config["model"]} submission (from kaggle notebook)'
#         )

Submission file saved as /kaggle/working/submission_RP3beta.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [50]:
import json

if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        # json.dump("topK=10, alpha=0.369533359594755, normalize_similarity=True, implicit=True", params_file)
        json.dump(study.best_trial.params, params_file)
        
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'TrainedModels/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/RP3betaRecommender/best_params_RP3beta.json' created successfully.


Save the history of the tuned model.

In [51]:
if config['save_github']:
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'TrainedModels/{config["model"]}Recommender/Submission/'\
                    f'submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)'
            )

File 'TrainedModels/RP3betaRecommender/Submission/submission_RP3beta.csv' created successfully.


In [52]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'TrainedModels/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)'
    )

File 'TrainedModels/RP3betaRecommender/history_RP3beta.db' created successfully.
